# CNN example

## MNIST dataset

라이브러리

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader # 배치 사이즈로 묶어서 전달

from tqdm.notebook import tqdm

하이퍼파라미터 설정

In [2]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

데이터 로드

In [6]:
# mnist 데이터셋을 다운할 때 에러가 뜬다면
from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [7]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), 
                         target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), 
                        target_transform=None, download=True)

train_loader = DataLoader(mnist_train, batch_size=batch_size,
                        shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                          shuffle=True, num_workers=2, drop_last=True)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


합성곱 모델

In [8]:
# nn.Module:
# 이미 torch.nn 클래스에 신경망 관련 많은 함수가 정의되어 있을 것이고, 
# 이것을 다 상속받아서 사용할 예정
class CNN(nn.Module): 
    
    
    # 처음에 클래스가 만들어질 때 어떤 값을 설정할지
    # 특별한 기능이 있음
    # 선언할 때 무조건 처음에 실행되는 것 (예약어)
    def __init__(self): 
        # nn.Module을 초기화하는 역할
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,200),
            nn.ReLU(),
            nn.Linear(200,10)
        )
        
    def forward(self,x): # self는 클래스 객체.forward 이런 식으로 쓰기위한 것
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

손실 함수 & 최적화 함수

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss() # 손실 함수
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 최적화할 변수로 매개변수를 사용하겠다!

학습

In [10]:
loss_arr = []
for i in tqdm(range(num_epoch)): # num_epoch 만큼 반복
    for j,[image,label] in tqdm(enumerate(train_loader)): # 배치마다 실행
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad() # 지난번에 계산했던 기울기를 0으로 초기화
        
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward() # 매개변수에 대한 기울기가 계산됨
        
        optimizer.step() # 매개변수 업데이트
        
        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3019, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.2405, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.2058, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0512, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0875, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0847, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0301, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0299, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0444, device='cuda:0', grad_fn=<NllLossBackward>)



tensor(0.0780, device='cuda:0', grad_fn=<NllLossBackward>)




테스트 데이터 검증

In [11]:
correct = 0
total = 0

with torch.no_grad(): # 기울기를 계산하지 않음
    for image,label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output,1) # 최댓값과 그 인덱스를 구함
        
        total += label.size(0)
        correct += (output_index == y_).sum().float() # 예측 라벨과 실제 라벨이 동일하면 더해줌
        
    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 98.79808044433594


# RNN

## hello pytorch. how long can a rnn cell remember?

라이브러리

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import numpy as np

하이퍼파라미터

In [64]:
n_hidden = 50
lr = 0.01
epochs = 1000

sentence = "hello pytorch. how long can a rnn cell remember?"

chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_chars = len(char_list)

문장 -> 원-핫 벡터

In [65]:
def string_to_onehot(string):
    
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=n_chars, dtype=int)
        zero[idx]=1
        
        start = np.vstack([start,zero])
    
    output = np.vstack([start,end])
    
    return output

원-핫 벡터 -> 문장

In [66]:
def onehot_to_word(onehot_1):

    # 텐서를 입력으로 받아 넘파이 배열로 바꿔준다.

    onehot = torch.Tensor.numpy(onehot_1)

    # one-hot 벡터의 최댓값(=1) 위치 인덱스로 문자를 찾는다.

    return char_list[onehot.argmax()]

모델

In [67]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.i2h = nn.Linear(input_size, hidden_size)
    self.h2h = nn.Linear(hidden_size, hidden_size)
    self.i2o = nn.Linear(hidden_size, output_size)
    self.act_fn = nn.Tanh()

  def forward(self, input, hidden):
    hidden = self.act_fn(self.i2h(input) + self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(1, self.hidden_size)

In [68]:
rnn = RNN(n_chars, n_hidden, n_chars) # 35, 35, 35

손실함수 & 최적화 함수

In [69]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

학습

In [70]:
## training

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()

    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1,:]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1),target.view(-1))
        total_loss += loss
        input_ = output

    total_loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print(f'Epoch {i}\'s loss: {total_loss.item()}')

Epoch 0's loss: 2.273893117904663
Epoch 100's loss: 0.027847835794091225
Epoch 200's loss: 0.004503707867115736
Epoch 300's loss: 0.0012772215995937586
Epoch 400's loss: 0.0005582173471339047
Epoch 500's loss: 0.0003110026300419122
Epoch 600's loss: 0.00014979140541981906
Epoch 700's loss: 0.00014062249101698399
Epoch 800's loss: 0.0011528696632012725
Epoch 900's loss: 2.6947560400003567e-05


테스트

In [71]:
start = torch.zeros(1,len(char_list))
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
        
print(output_string)

hello pytorch. how long can a rnn cell rnnepb py
